In [ ]:
#Cell_1.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter('ignore')

import plotly

plt.rcParams['font.family'] = 'Noto Sans' # for Japanese in graph (Win)

In [ ]:
#Cell_2.
from greykite.algo.changepoint.adalasso.changepoint_detector import ChangepointDetector
from greykite.algo.forecast.silverkite.constants.silverkite_column import SilverkiteColumn
from greykite.common.features.timeseries_features import get_available_holidays_across_countries
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [ ]:
#Cell_3.
csv_in="8306_2018.csv"
df=pd.read_csv(csv_in,skiprows=1,header=0,delimiter=',',encoding="shift-jis")
print(df.shape) #確認
print(df.head()) #確認
print(type(df['日付'][0]))
df['日付']=pd.to_datetime(df['日付'])
print(type(df['日付'][0]))

In [ ]:
#Cell_4.
df_d=df.copy()
df_d['経過日数']=df_d['日付']
df_d['経過日数']-=df_d['経過日数'][0]
print(type(df_d['経過日数'][0]))
df_d['経過日数']/=np.timedelta64(1,'D')
print(type(df_d['経過日数'][0]))
print(df_d.head())

In [ ]:
#Cell_5.
df_d['week']=0
print(df_d.shape)
display(df_d.head())

In [ ]:
#Cell_6.
df_d['week'] = df_d['経過日数']//7 + 1

In [ ]:
#Cell_7.
display(df_d.head())
display(df_d.tail())

In [ ]:
#Cell_8.
#Extract 2018 and onward.
interest = pd.read_csv('d1-5-5.csv',skiprows=525,encoding='cp932')
display(interest.head())

In [ ]:
#Cell_9.
interest=interest.iloc[:,0:4]
interest.columns=['week','US','EUR','JP']
display(interest.head())
print(interest.info())
for i in range(23):
    interest.loc[i,'week']=i+1
display(interest.head(23))


In [ ]:
#Cell_10.
interest2 = interest.iloc[0:23,:]
display(interest2)

In [ ]:
#Cell_11.
df2= pd.merge(df_d,interest2,on='week',how='left')
df2=df2.drop('経過日数',axis=1)
display(df2.head())
display(df2.tail())

In [ ]:
#Cell_12.
metadata = MetadataParam(
    time_col="日付", # 日付の列名
    value_col="終値", # 値の列名
    freq="D", # 日付の単位
    train_end_date=pd.datetime(2018, 5, 20)
)

In [ ]:
#Cell_13.
regressors=dict(
    regressor_cols=[
        ["出来高", "US"]
    ]
)

In [ ]:
#Cell_14.
yearly_seasonality_order = 10
weekly_seasonality_order = 5
seasonality = {
    "yearly_seasonality": yearly_seasonality_order,
    "quarterly_seasonality": False,
    "monthly_seasonality": False,
    "weekly_seasonality": weekly_seasonality_order,
    "daily_seasonality": False
}

In [ ]:
#Cell_15.
growth = {
 "growth_term": "linear"
}
changepoints = {
 "changepoints_dict": dict(
     method="auto",
     yearly_seasonality_order=10,
     regularization_strength=0.5,
     resample_freq="7D",
     potential_changepoint_n=25,
     yearly_seasonality_change_freq="365D",
     no_changepoint_distance_from_end="365D"
 )
}

In [ ]:
#Cell_16.
model_components = ModelComponentsParam(
    seasonality=seasonality,
    growth=growth,
    #events=events,
    #changepoints=changepoints,
    autoregression=None,
    regressors=regressors,
    uncertainty={
     "uncertainty_dict": "auto",
    },
    custom={
     "fit_algorithm_dict": {
         "fit_algorithm": "ridge",
     },
     #"extra_pred_cols": extra_pred_cols
    }
)

In [ ]:
#Cell_17.
forecaster = Forecaster()
result = forecaster.run_forecast_config(
    df=df2,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=14,
        coverage=0.95, # 予測区間を95%に設定
        metadata_param=metadata,
        model_components_param=model_components
    )
)

In [ ]:
#Cell_18.
print(result.model[-1].summary(max_colwidth=50))

In [ ]:
#Cell_19.
backtest = result.backtest
fig = backtest.plot()
plotly.io.show(fig)

In [ ]:
#Cell_20.
fig = result.forecast.plot_components()
plotly.io.show(fig)

In [ ]:
#Cell_21.
model = ChangepointDetector()
res = model.find_trend_changepoints(
    df=df2, # データフレーム
    time_col="日付", # 日付の列名
    value_col="終値", # 値の列名
    yearly_seasonality_order=0, # 年周期を表現するためのフーリエ級数の次数(今回は年周期を仮定してないので0)
    regularization_strength=0.5, # adaptive lassoの罰則の強さ(0~1の値)
    resample_freq="7D", # データを括る粒度
    potential_changepoint_n=20, # 変化点の数、値を大きくすると計算負荷が高くなる
    yearly_seasonality_change_freq=None, # 年周期の値が変わる間隔(今回は年周期ないので無視)
    no_changepoint_distance_from_end="14D", # ラスト14日は変化点なし
    actual_changepoint_min_distance="7D" # 変化点どうしの感覚は最低7日空ける
)

fig = model.plot(
    observation=True,
    trend_estimate=False,
    trend_change=True,
    yearly_seasonality_estimate=False,
    adaptive_lasso_estimate=True,
    plot=False)
plotly.io.show(fig)

In [ ]:
#Lets try a customized model.
#Cell_22.
growth = {
    "growth_term": "linear" # 線形トレンド
}

changepoints = {
    "changepoints_dict": {
        "method": "auto",
        "yearly_seasonality_order": 0,
        "regularization_strength": 0.5,
        "resample_freq": "7D",
        "potential_changepoint_n": 20,
        "yearly_seasonality_change_freq": None,
        "no_changepoint_distance_from_end": "14D",
        "actual_changepoint_min_distance": "7D"
    }
}

seasonality = {
    "yearly_seasonality": False,
    "quarterly_seasonality": False,
    "monthly_seasonality": False,
    "weekly_seasonality": 2, # 週周期の次数
    "daily_seasonality": False
}

events = {
    "holiday_lookup_countries": None # 祝日なし
}

model_components = ModelComponentsParam(
    seasonality=seasonality,
    growth=growth,
    changepoints=changepoints,
    autoregression={
        "autoreg_dict": None # 自己回帰の項なし
    },
    events=events,
    regressors=regressors,
    lagged_regressors={
        "lagged_regressor_dict": None # 説明変数のラグも当然なし
    },
    custom={
        "fit_algorithm_dict": {
            "fit_algorithm": "ridge"
        },
        "feature_sets_enabled": {
            SilverkiteColumn.COLS_DAY_OF_WEEK: False, # 曜日ダミーの有無
            SilverkiteColumn.COLS_TREND_WEEKEND: False, # 平日/週末とトレンドとの交互作用の有無
            SilverkiteColumn.COLS_TREND_DAY_OF_WEEK: False, # 曜日とトレンドとの交互作用の有無
            SilverkiteColumn.COLS_TREND_WEEKLY_SEAS: False # 週周期の変化を許容するかどうか
        }
    },
    uncertainty={
        "uncertainty_dict": {
            "uncertainty_method": "simple_conditional_residuals",
            "params": {
                "quantiles": [0.025, 0.975],
                "conditional_cols": ["month"], # 予測区間を月ごとに計算する
                "quantile_estimation_method": "normal_fit"
            }

        },
    }
)

In [ ]:
#Cell_23.
custom_result = forecaster.run_forecast_config(
    df=df2,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=14,
        coverage=0.95,  # 95% prediction intervals
        model_components_param=model_components,
        metadata_param=metadata
    )
)

In [ ]:
#Cell_24.
backtest = custom_result.backtest
fig = backtest.plot()
plotly.io.show(fig)

In [ ]:
#Cell_25.
print(custom_result.model[-1].summary(max_colwidth=50))

In [ ]:
#Cell_26.
fig = custom_result.forecast.plot_components()
plotly.io.show(fig)

In [ ]:
#Cell_27.
custom_result.backtest.test_evaluation